# Separate FloSup data into smaller sections using CSV or shapefile and Arcpy
## Write to Csv, return number of photos per section
### Jin-Si R. Over - jover@usgs.gov
### v1 -  7/23/2020
### v2 - 8/24/2020
### v3 - 8/27/2020 - added splitting sections and new zip names
### v4 - 9/3/2020 - added ability to work without using arc, only use cell 4 OR 5, not both

To Do: 
1. automate shapefiles and maps?
2. If split, write out gdb files of the split csvs for mapping?
3. Tranpose PhotoCount.csv when its output

In [1]:
#ArcDesktop - 10.7 running python 2.7
#Note, should run in python 3...

import os
import sys
try:
    import archook #The module which locates arcgis
    archook.get_arcpy()
    import arcpy
except ImportError:
    print ("can't find arcpy")
    
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from scipy import interpolate, signal
import glob
import pandas as pd
import csv
import copy

from arcpy import env
from arcpy.sa import *
import arcpy.cartography as CA

arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")
env.overwriteOutput = True
arcpy.overwriteOutput = True

In [2]:
#EDIT THIS CELL ONLY

#Variables embedded in cells below
release = 'postFlorence'  #Each flight is either post or pre 'Hurricane name' and then month abrv if more than more post flight. 
yearMonth = '2018-10'     #Change to reflect the flight date, if spanning over multiple months, choose the first date. 
state = '_NC_'            #Change for flight area name
step = 0.05               #Change to set range of latitude "bins" (0.05 decimal degrees is 3 minutes)
llb = 33.8                #lower latitudinal boundary, 33.8 is cape fear, 33.7 is the lowest part of NC
ulb = 36.2                #upper latutudinal boundary, 36.2 encompases up to around Duck, 36.55 is VA/NC border
l = ['a','b','c','d','e'] #Labels for split csvs. 
limit = 750               #Max number of photos in a zip file, 750 is roughly 20 GB if each photo is ~20 MB

#Can change to decide which columns to keep in final csvs
keepCol = ['Station','UTCTime', 'Latitude', 'Longitude', 'Easting','Northing'] 

#Working Folder on Desktop
wkspc = r"C:/Users/jover/Working/02_USGS/08_Aerial_Metadata/"                  

#Metadata Folder - based on imgery to be released
export_path = r"C:/Users/jover/Working/02_USGS/08_Aerial_Metadata/" + release

print("Setting workspace to the following directory: " + wkspc)


# Set paths and names
locRaw              = wkspc + '01_RawData/'  #Csv files for each flight
locgdb 				= wkspc + release + '/' + release + '_Photo_Locations.gdb/'		# Geodatabase location
locshp				= wkspc + '02_Shapefiles/'			# Shapefiles named as release + "PhotoLoc_all"- should be from a merged csv for each flight if on multiple days)
loccsv              = wkspc + release + '/FlightZips/' #Place the names for each section
env.workspace       = locgdb

Setting workspace to the following directory: C:/Users/jover/Working/02_USGS/08_Aerial_Metadata/


In [3]:
#Set up floating ranges to call in a loop
first = np.arange(llb, ulb, step)                  #Lower bounding range of latitude floats that I want to separate out
second = np.arange(llb + step, ulb + step, step)   #Upper bounding range based on step
#zips = range(0,len(first),1)                       #Just an index for naming the output csvs

#Test naming conventions for the zip files - this may not work if step changes?
f = round(first[1],2)             #Round the step to make sure increments by 3
print(f)
print(int(str(f)[0:2]))           #Call the first two numbers (degrees)
print(float(str(f)[2:5])*60+100)  #Call the second two numbers as a decimal and convert to minutes (add 100 to avoid zero problems)
print(yearMonth + state + str(f)[0:2] + str(float(str(f)[2:5])*60+100)[1:3] + 'N' + '.csv') #Put it all together



33.85
33
151.0
2018-10_NC_3351N.csv


In [5]:
#IF USING THIS CELL DO NOT RUN THE NEXT ONE - IT DOES THE SAME THING IF A SHAPEFILE IS AVAILABLE

#Loop to select out file names from Flights by step using the csv in the 01_RawData folder

#Which csv to read
df = pd.read_csv(locRaw + release + '_Photo_Locations_all.csv')

#Loop for latitude intervals
for j in range(len(first)):
    temp = df[(df['Latitude'] >= first[j]) & (df['Latitude'] < second[j])]
    
    #Naming conventions
    f = round(first[j],2)
    zname = yearMonth + state + str(f)[0:2] + str(float(str(f)[2:5])*60+100)[1:3] + 'N'  #Output name - change parts in cell above
    
    #Create the new csv
    temp.to_csv(loccsv + zname + '.csv')
    

In [7]:
#DO NOT RUN THIS IF ALREADY USING PREVIOUS CELL - WILL OVERWRITE FOR NO REASON, note this cell will also create auxilary files


#Loop to select out file names from Flights by .05 (or whatever step specifed) decimal degrees using a shapefile

for i in range(len(first)):
    #Make a working layer to access from a point shapefile containing all the locations/data
    #The shapefile should be all the Flight Locations csv files merged with only one heading
    arcpy.MakeFeatureLayer_management(locshp + release + "_PhotoLoc_all.shp", release + '_PhotoLoc_lyr')
    
    #Select from the layer by the latitude attribute and the ranges specified in the cell above
    sectionPhotos = arcpy.SelectLayerByAttribute_management(release + '_PhotoLoc_lyr',"NEW_SELECTION","Latitude >= " + str(first[i]) + " AND Latitude < " + str(second[i]))
    
    #Creates copy of the selected features and puts them into a gdb feature class that is overwritten each time
    arcpy.CopyFeatures_management(sectionPhotos, locgdb + "overwrite")
    
    #Takes the file being overwritten and makes it into a csv with an incremental name based off of the date, state, and latitude 4 digit code
    # TableToTable takes three inputs, feature class, output location, output name.
    f = round(first[i],2)
    zname = yearMonth + state + str(f)[0:2] + str(float(str(f)[2:5])*60+100)[1:3] + 'N'  #Output name - change parts in cell above
    
    arcpy.TableToTable_conversion(locgdb + "overwrite", loccsv, zname + '.csv')

In [6]:
#Get rid of extraneous columns in the newly csvs

#Walk through the created csv, read them using pandas and then rewrite it only using the columns we want to keep
for f in glob.glob(loccsv + "*.csv"):
    fn = pd.read_csv(f)
    new_f = fn[keepCol]
    new_f.to_csv(f)

In [12]:
#Split CSVs in multiple -a, -b, etc sections if length is greater than 750 (or set the limit variable) images. Deletes the pre-split csv. 
#Note that the splits are based purely on math, they aren't by date or smaller sections of the latitude. 
os.chdir(loccsv)

for fn in glob.glob('*N.csv'):      #Walk through every csv other than the Photo_count if its in the same folder 
    with open(fn) as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        rows = [row for row in reader]
        row_count = len(rows)             #Count number of rows in the csv
        
        #Do some basic math to determine how many csvs will need to be made, at the moment the code only goes up to five splits (a-e)
        d2 = int(round(row_count/2,0))
        d3 = int(round(row_count/3,0))
        d4 = int(round(row_count/4,0))
        d5 = int(round(row_count/5,0))
        #print(row_count)
        
        if row_count < limit:
            print('No split necessary for %s' % f)
        
        elif  d2 < 750:
            pages = []
            print('There are %d photos, splitting into two sections with %d in each...' % (row_count,d2))
            start_index = 0
            while start_index < row_count:
                pages.append(rows[start_index: start_index + (d2+2)])   #add 2 to prevent issues with an odd split adding an extra csv with only one line
                start_index += d2+2
            for i, page in enumerate(pages):
                with open('{}-{}.csv'.format(fn[0:16], l[i]), 'wb') as outfile:  #Name the split csvs - if naming convention changes need to change the [0:16]
                    writer = csv.DictWriter(outfile, fieldnames = header)        #Keep headers when splitting
                    writer.writeheader()
                    for row in page:
                          writer.writerow(row)
            #If split, close the file and delete it before moving on
            f.close()
            os.remove(fn)
        elif d3 < limit:
            pages = []
            print('There are %d photos, splitting into three sections with %d in each...' % (row_count,d3))
            start_index = 0
            while start_index < row_count:
                pages.append(rows[start_index: start_index + (d3+2)])
                start_index += d3+2
            for i, page in enumerate(pages):
                with open('{}-{}.csv'.format(fn[0:16], l[i]), 'wb') as outfile:
                    writer = csv.DictWriter(outfile, fieldnames = header)
                    writer.writeheader()
                    for row in page:
                          writer.writerow(row)
            f.close()
            os.remove(fn)         
        elif d4 < limit:
            pages = []
            print('There are %d photos, splitting into four sections with %d in each...' % (row_count,d4))
            start_index = 0
            while start_index < row_count:
                pages.append(rows[start_index: start_index + (d4+2)])
                start_index += d4+2
            for i, page in enumerate(pages):
                with open('{}-{}.csv'.format(fn[0:16], l[i]), 'wb') as outfile:
                    writer = csv.DictWriter(outfile, fieldnames = header)
                    writer.writeheader()
                    for row in page:
                          writer.writerow(row)
            f.close()
            os.remove(fn)              
        elif d5 < limit:
            pages = []
            print('There are %d photos, splitting into five sections with %d in each...' % (row_count,d5))
            start_index = 0
            while start_index < row_count:
                pages.append(rows[start_index: start_index + (d5+2)])
                start_index += d5+2
            for i, page in enumerate(pages):
                with open('{}-{}.csv'.format(fn[0:16], l[i]), 'wb') as outfile:
                    writer = csv.DictWriter(outfile, fieldnames = header)
                    writer.writeheader()
                    for row in page:
                          writer.writerow(row)
            f.close()
            os.remove(fn)               
        else:
            print('Manually split because this is getting ridiculous')
                            
                    
        

TypeError: 'int' object is not callable

In [11]:
#Write out how many photos are in each file and write out as a csv - https://stackoverflow.com/questions/31616217/python-script-to-count-num-lines-in-all-files-in-directory
#The csv needs to be transposed though...

#os.chdir('C:/Users/jover/Working/02_USGS/01b_OBX_Florence/01_Flight_Zip_Sections/')
#os.chdir('C:/Users/jover/Working/02_USGS/01b_OBX_Florence/001_FlightZips/')
os.chdir(loccsv)
names={}
for fn in glob.glob('*N*.csv'):                          #Walk through every csv other than the  (fixed so it won't count the Photo_count if its in the same folder) in the zipped sections
    with open(fn) as f:
        names[fn]=sum(1 for line in f if line.strip())-1 #counts the lines, minus the header and sums them for each csv
        
with open('Photo_count.csv','wb') as f:                #Save as a csv
    w = csv.DictWriter(f,names.keys());
    w.writeheader();
    w.writerow(names)                                  #First row is the names of the csvs, second is the count
    
